In [10]:
# Импорт модулей и библиотек

import numpy as np
import pandas as pd

from imblearn.over_sampling import SMOTE
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold

from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegressionCV
from sklearn.metrics import f1_score, precision_score, accuracy_score, recall_score

In [3]:
# Чтение dataset.v.3.2.csv
dataset = pd.read_csv('_data/dataset.v.3.2.csv', sep=';')
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35227 entries, 0 to 35226
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   ticker       35227 non-null  object 
 1   year         35227 non-null  int64  
 2   month        35227 non-null  int64  
 3   day          35227 non-null  int64  
 4   title        35227 non-null  object 
 5   content      35227 non-null  object 
 6   T            35227 non-null  float64
 7   a1           35227 non-null  float64
 8   a2           35227 non-null  float64
 9   title_cln    35227 non-null  object 
 10  content_cln  35227 non-null  object 
dtypes: float64(3), int64(3), object(5)
memory usage: 3.0+ MB


In [4]:
# Подготовка данные, вычисление класса Y

period = 30
dataset = dataset[dataset['T'] == period]
dataset = dataset[dataset['a2'].notnull()]

# Y=1, если котировка имела нисходящий тренд
dataset['Y'] = (dataset['a2'] < 0) * 1 

r0, r1 = dataset['Y'].value_counts() # кол-во примеров классов
w0, w1 = 1 - r0/len(dataset), 1 - r1/len(dataset) # веса классов

print(f'Количество записей: {len(dataset)}')
print(f'Класс 0 (рост котировки): {r0} примеров, вес: {round(w0, 2)}')
print(f'Класс 1 (падение котировки): {r1} примеров, вес: {round(w1, 2)}')

Количество записей: 35227
Класс 0 (рост котировки): 20418 примеров, вес: 0.42
Класс 1 (падение котировки): 14809 примеров, вес: 0.58


In [21]:
# Получение train и test-выборок

min_df = 1
train_size = 0.9
test_size = 0.1

train, test = train_test_split(dataset, test_size=test_size, random_state=42)

X_train, X_test, y_train, y_test = train['title'], test['title'], train['Y'], test['Y']

vectorizer = CountVectorizer(min_df=min_df)
vectorizer.fit(X_train)
X_train = vectorizer.transform(X_train)
X_test = vectorizer.transform(X_test)

# Балансировка SMOTE
smote = SMOTE(sampling_strategy='auto', k_neighbors=5, random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

print(f'Обучающая выборка: {X_train_resampled.shape[0]} записей')
print(f'Тестовая выборка: {X_test.shape[0]} записей')
print(f'Атрибутов: {X_test.shape[1]}')

print(f'После балансировки:\nКласс 0: {y_train_resampled.value_counts()[0]} примеров')
print(f'Kласс 1: {y_train_resampled.value_counts()[1]} примеров')

Обучающая выборка: 36762 записей
Тестовая выборка: 3523 записей
Атрибутов: 22493
После балансировки:
Класс 0: 18381 примеров
Kласс 1: 18381 примеров


In [38]:
# Логистическая регрессия без тюнинга С-параметра

lr=LogisticRegression(C = 5,
                      max_iter = 700,
                      class_weight = {1: 0.5, 0: 0.5})

model = lr.fit(X_train_resampled, y_train_resampled)

y_pred_proba = model.predict_proba(X_test)
y_pred = model.predict(X_test)

In [39]:
print(f'Accuracy = {accuracy_score(y_test, y_pred)}')
print(f'Recall = {recall_score(y_test, y_pred)}')
print(f'Precision = {precision_score(y_test, y_pred)}')
print(f'F1 = {f1_score(y_test, y_pred)}')

Accuracy = 0.5975021288674425
Recall = 0.5753701211305519
Precision = 0.5207064555420219
F1 = 0.5466751918158568


In [37]:
# Логистическая регрессия с автоматическим тюнингом С-параметра

skf = StratifiedKFold(n_splits=2, shuffle=True, random_state=42)

c_range = np.logspace(-2, 3, 100)

model = LogisticRegressionCV(Cs=c_range, cv=skf, verbose=0, n_jobs=-1, scoring = 'accuracy')
model.fit(X_train_resampled, y_train_resampled)

y_pred = model.predict(X_test)

In [36]:
print(f'Accuracy = {accuracy_score(y_test, y_pred)}')
print(f'Recall = {recall_score(y_test, y_pred)}')
print(f'Precision = {precision_score(y_test, y_pred)}')
print(f'F1 = {f1_score(y_test, y_pred)}')

Accuracy = 0.603179108714164
Recall = 0.5746971736204576
Precision = 0.5271604938271605
F1 = 0.549903412749517
